<a href="https://colab.research.google.com/github/rufous86/studies/blob/main/recommender_systems/RecSys_on_practice_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/competitions/recsys-in-practice/data?select=train_joke_df.csv

In [1]:
! pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
! cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c recsys-in-practice
! unzip recsys-in-practice.zip -d .

Mounted at /content/drive
 45% 5.00M/11.2M [00:00<00:00, 35.6MB/s]
100% 11.2M/11.2M [00:00<00:00, 58.8MB/s]
Archive:  recsys-in-practice.zip
  inflating: ./sample_submission.csv  
  inflating: ./test_joke_df_nofactrating.csv  
  inflating: ./train_joke_df.csv     


In [2]:
! pip install implicit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 40.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import scipy.sparse as sp
from scipy.sparse import csr_matrix

import implicit

/usr/local/lib/python3.9/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [4]:
df_train = pd.read_csv('train_joke_df.csv')
df_train

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60
...,...,...,...
1448359,22604,26,2.82
1448360,22255,36,-1.94
1448361,21056,40,-9.56
1448362,12328,97,0.87


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448364 entries, 0 to 1448363
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   UID     1448364 non-null  int64  
 1   JID     1448364 non-null  int64  
 2   Rating  1448364 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 33.2 MB


In [6]:
df_train.describe()

,UID,JID,Rating
count,1.448364e+06,1.448364e+06,1.448364e+06
mean,1.252160e+04,4.347179e+01,8.817123e-01
std,7.198297e+03,2.619797e+01,5.234737e+00
min,1.000000e+00,1.000000e+00,-9.950000e+00
25%,6.283000e+03,2.100000e+01,-2.960000e+00
50%,1.255400e+04,4.200000e+01,1.500000e+00
75%,1.875000e+04,6.300000e+01,5.100000e+00
max,2.498300e+04,1.000000e+02,1.000000e+01


In [10]:
df_train['UID'].nunique(), df_train['JID'].nunique(), df_train['Rating'].nunique()

(24983, 100, 412)

In [ ]:
data = df_train['Rating']
row = df_train['UID']
col = df_train['JID']
# convert DataFrame to COO sparse matrix
user_item_data = sp.coo_matrix((data, (row, col))).tocsr()

In [ ]:
model = implicit.als.AlternatingLeastSquares(
                                    factors=500,
                                    regularization=1.0,
                                    alpha=1.0,
                                    calculate_training_loss=True, 
                                    iterations=30)
model.fit(user_item_data)

In [ ]:
rec_dict = {
    'UID': [],
    'predictions': []    
}

for idx, row in private_test_df.iterrows():
    cookie_id = row['cookie_id']
    userid = dict_cookie_id[cookie_id]
    recommendations = model.recommend(userid, user_item_data[userid], N=5)[0]
    rec_dict['cookie_id'].append(cookie_id)
    rec_dict['predictions'].append(list(recommendations))

recommendation_df = pd.DataFrame(rec_dict)
recommendation_df